### Importing Libraries

Importing necessary libraries for pre-processing 

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

### Load E Commerce Dataset

Load dataset csv using pandas 

In [2]:
Ecommerce = pd.read_csv('ecommerce.csv',low_memory=False)
pd.set_option('display.max_columns', None)
#Ecommerce['Order Date'] = pd.to_datetime(Ecommerce['Order Date'], format = '%m/%d/%y')
#Ecommerce['Ship Date'] = pd.to_datetime(Ecommerce['Ship Date'], format = '%m/%d/%y')
Ecommerce.head(5)

,Order ID,Order Date,Ship Date,Aging,Ship Mode,Product Category,Product,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Customer ID,Customer Name,Segment,City,State,Country,Region,Months
0,AU-2015-1,11/9/15,11/17/15,8.0,First Class,Auto & Accessories,Car Media Players,$140.00,2,0.05,$46.00,$4.60,Medium,LS-001,Lane Daniels,Consumer,Brisbane,Queensland,Australia,Oceania,Nov
1,AU-2015-2,6/30/15,7/2/15,2.0,First Class,Auto & Accessories,Car Speakers,$211.00,3,0.03,$112.00,$11.20,Medium,IZ-002,Alvarado Kriz,Home Office,Berlin,Berlin,Germany,Central,Jun
2,AU-2015-3,12/5/15,12/13/15,8.0,First Class,Auto & Accessories,Car Body Covers,$117.00,5,0.01,$31.20,$3.10,Critical,EN-003,Moon Weien,Consumer,Porirua,Wellington,New Zealand,Oceania,Dec
3,AU-2015-4,5/9/15,5/16/15,7.0,First Class,Auto & Accessories,Car & Bike Care,$118.00,2,0.05,$26.20,$2.60,High,AN-004,Sanchez Bergman,Corporate,Kabul,Kabul,Afghanistan,Central Asia,May
4,AU-2015-5,7/9/15,7/18/15,9.0,First Class,Auto & Accessories,Tyre,$250.00,1,0.04,$160.00,$16.00,Critical,ON-005,Rowe Jackson,Corporate,Townsville,Queensland,Australia,Oceania,Jul


### Dataset Profiling

In [3]:
print("Dataset dimension :", Ecommerce.ndim)
print("Dataset shape :", Ecommerce.shape)
print("Dataset size :", Ecommerce.size)

Dataset dimension : 2
Dataset shape : (51290, 21)
Dataset size : 1077090


Dataset has 2 dimensions, comprises of 51290 observations (rows) and 21 features (columns), multiplying the two numbers, we have a total 1077090 number of elements of our dataset.

In [4]:
Ecommerce_types = Ecommerce.dtypes.reset_index()
Ecommerce_types.columns = ['Features','Data Types']
Ecommerce_types

,Features,Data Types
0,Order ID,object
1,Order Date,object
2,Ship Date,object
3,Aging,float64
4,Ship Mode,object
5,Product Category,object
6,Product,object
7,Sales,object
8,Quantity,object
9,Discount,object


At the moment, Dataset is dominated by object / string Dtype with total of 20 variables, while the other is numeric.

### Dataset Inspection
### Dataset Missing and Blank Value Checking

Dataset checking for missing and blank values.

In [5]:
missing_data = Ecommerce.isnull().sum().reset_index()
missing_data.columns = ['Features','Missing Values']
blank_data = Ecommerce.apply(lambda x: x == ' ').sum().reset_index()
blank_data.columns = ['Features','Blank Values']
merged_data = pd.merge(missing_data,blank_data, on=['Features'])
merged_data['Filling Factor'] = (Ecommerce.shape[0]-missing_data['Missing Values'] + Ecommerce.shape[0]-blank_data['Blank Values'])/Ecommerce.shape[0]*50
merged_data.sort_values(by='Filling Factor',ascending=True)

,Features,Missing Values,Blank Values,Filling Factor
12,Order Priority,2,0,99.998050
1,Order Date,1,0,99.999025
3,Aging,1,0,99.999025
15,Segment,1,0,99.999025
8,Quantity,1,0,99.999025
13,Customer ID,1,0,99.999025
19,Region,1,0,99.999025
0,Order ID,0,0,100.000000
18,Country,0,0,100.000000
17,State,0,0,100.000000


There are several features with a or two missing value(s) within, while no blank values detected in all features. Dataset cleaning process will be performed to features having a missing values. The process done depends on how the missing values will be filled / replaced. Before we perform data cleaning it is best to check duplicated data(s).

### Data Duplication Checking

In [6]:
Ecommerce.duplicated().sum()

0

It is seen that no duplicated data detected.

### Selected Features Unique Value Check

This is done to cross-check the values within each features, to see if there are any abnormal / anomaly. I noticed there are features with very odd value, for example, 'N4rth' and 'S03th'are included in the Region feature, which as it name sugggest it should represents a region / parts of the world and not some random misspelling of the sort. 

### Order Priority

In [7]:
OP_count = Ecommerce['Order Priority'].value_counts().reset_index()
OP_count.columns = ['Order Priority','Total Count']
OP_count.sort_values(by='Total Count',ascending=False)

,Order Priority,Total Count
0,Medium,29433
1,High,15499
2,Critical,3932
3,Low,2424


### Aging

In [8]:
AG_count = Ecommerce['Aging'].value_counts().reset_index()
AG_count.columns = ['Aging','Total Count']
AG_count.sort_values(by='Total Count',ascending=False).tail(3)

,Aging,Total Count
8,2.0,4815
9,9.0,4674
10,10.5,1


### Segment

In [9]:
SG_count = Ecommerce['Segment'].value_counts().reset_index()
SG_count.columns = ['Segment','Total Count']
SG_count.sort_values(by='Total Count',ascending=False).tail(3)

,Segment,Total Count
0,Consumer,26518
1,Corporate,15429
2,Home Office,9342


### Quantity

In [10]:
QT_count = Ecommerce['Quantity'].value_counts().reset_index()
QT_count.columns = ['Quantity','Total Count']
QT_count.sort_values(by='Total Count',ascending=False).tail(3)

,Quantity,Total Count
3,2,10209
4,5,10196
5,abc,1


### Region

In [11]:
RG_count = Ecommerce['Region'].value_counts().reset_index()
RG_count.columns = ['Region','Total Count']
RG_count.sort_values(by='Total Count',ascending=False).tail(3)

,Region,Total Count
12,Canada,384
13,4orth,1
14,So3th,1


### Country

In [12]:
CT_count = Ecommerce['Country'].value_counts().reset_index()
CT_count.columns = ['Country','Total Count']
CT_count.sort_values(by='Total Count',ascending=False).tail(3)

,Country,Total Count
144,Bahrain,2
145,South Sudan,2
146,Eritrea,2


### State

In [13]:
ST_count = Ecommerce['State'].value_counts().reset_index()
ST_count.columns = ['State','Total Count']
ST_count.sort_values(by='Total Count',ascending=False).tail(3)

,State,Total Count
1058,Pernik,1
1059,Elbasan,1
1093,Inhambane,1


### City

In [14]:
CY_count = Ecommerce['City'].value_counts().reset_index()
CY_count.columns = ['City','Total Count']
CY_count.sort_values(by='Total Count',ascending=False).tail(3)

,City,Total Count
3304,Gitarama,1
3303,Littleton,1
3635,Toul,1


### Discount

In [15]:
DS_count = Ecommerce['Discount'].value_counts().reset_index()
DS_count.columns = ['Discount','Total Count']
DS_count.sort_values(by='Total Count',ascending=False).tail(3)

,Discount,Total Count
3,0.01,10242
4,0.05,10172
5,xxx,1


### Sales

In [16]:
SL_count = Ecommerce['Sales'].value_counts().reset_index()
SL_count.columns = ['Sales Number','Total Count']
SL_count.sort_values(by='Total Count',ascending=False).tail(3)

,Sales Number,Total Count
37,$104.00,221
38,$199.00,221
39,0.xf,1


### Shipping Cost

In [17]:
SC_count = Ecommerce['Shipping Cost'].value_counts().reset_index()
SC_count.columns = ['Shipping Cost','Total Count']
SC_count.sort_values(by='Total Count',ascending=False).tail(3)

,Shipping Cost,Total Count
151,$6.80,7
152,$8.10,6
153,test,1


### Data Cleaning

### Order Priority Missing Value Handling

Filling missing value with the most frequent / occured value  in the feature.

In [18]:
# Replacing Nan to value type occurs more frequently than others
Ecommerce['Order Priority'] = Ecommerce['Order Priority'].fillna(value='Medium')

# Double Checking Missing and Blank values
OP_fill = Ecommerce['Order Priority'].isnull().sum()
OP_blnk = Ecommerce['Order Priority'].apply(lambda x : x == ' ').sum()

print('Order Priority missing value:', OP_fill , 'and blank value :', OP_blnk)

Order Priority missing value: 0 and blank value : 0


### Aging Missing Value Handling

The same as Order Priority, filling the missing value with the most frequent value in the feature.

In [19]:
# Replacing Nan to value type occurs more frequently than others
Ecommerce['Aging'] = Ecommerce['Aging'].fillna(value= 1)

# Double Checking Missing and Blank values
AG_fill = Ecommerce['Aging'].isnull().sum()
AG_blnk = Ecommerce['Aging'].apply(lambda x : x == ' ').sum()

print('Aging missing value:', AG_fill , 'and blank value :', AG_blnk)

Aging missing value: 0 and blank value : 0


Unique values cross checking.

In [20]:
# Changing anomaly value
Ecommerce['Aging'].replace([10.5],[1],inplace=True)

# Cross Checking its unique value
AG_count = Ecommerce['Aging'].value_counts().reset_index()
AG_count.columns = ['Aging','Total Count']
AG_count.sort_values(by='Total Count',ascending=False).tail(5)

,Aging,Total Count
5,10.0,4890
6,4.0,4875
7,8.0,4820
8,2.0,4815
9,9.0,4674


### Segment Missing Value Handling

The same as Order Priority, filling the missing value with the most frequent value in the feature.

In [21]:
# Replacing Nan to value type occurs more frequently than others
Ecommerce['Segment'] = Ecommerce['Segment'].fillna(value='Consumer')

# Double Checking Missing and Blank values
SG_fill = Ecommerce['Segment'].isnull().sum()
SG_blnk = Ecommerce['Segment'].apply(lambda x : x == ' ').sum()

print('Segment missing value:', SG_fill , 'and blank value :', SG_blnk)

Segment missing value: 0 and blank value : 0


### Region Missing Value Handling

To fill the missing value for this specific feature, i first locate which row the missing value lies, then i adjust / match it with the other rows which in the same country as said rows intersect. Italy it is, afterward i just fill the missing value with the region in which italy lies, The South Region.

In [22]:
filt3 = Ecommerce['Country'] == 'Italy'
Ecommerce.loc[filt3].head(3)

,Order ID,Order Date,Ship Date,Aging,Ship Mode,Product Category,Product,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Customer ID,Customer Name,Segment,City,State,Country,Region,Months
117,AU-2015-118,8/16/15,8/17/15,1.0,First Class,Auto & Accessories,Car Media Players,$140.00,1,0.04,$54.40,$5.40,High,RI-00118,Ayala Molinari,Consumer,Turin,Piedmont,Italy,NaN,Aug
142,AU-2015-143,3/23/15,3/26/15,3.0,First Class,Auto & Accessories,Car Seat Covers,$114.00,4,0.05,$11.20,$1.10,Critical,EY-00143,Robertson Coakley,Consumer,Barletta,Apulia,Italy,South,Mar
204,AU-2015-205,4/6/15,4/16/15,10.0,First Class,Auto & Accessories,Car Mat,$54.00,2,0.04,$27.00,$2.70,Critical,RF-00205,Mejia Waldorf,Corporate,Pozzuoli,Campania,Italy,South,Apr


In [23]:
# Replacing Nan to South
Ecommerce['Region'] = Ecommerce['Region'].fillna(value='South')

# Double Checking Missing and Blank values
RG_fill = Ecommerce['Region'].isnull().sum()
RG_blnk = Ecommerce['Region'].apply(lambda x : x == ' ').sum()

print('Region missing value:', RG_fill , 'and blank value :', RG_blnk)

Region missing value: 0 and blank value : 0


Cross checking unique value.

In [24]:
# Locating anomaly value
filt5 = (Ecommerce['Region'] == '4orth') | (Ecommerce['Region'] == 'So3th')
Ecommerce.loc[filt5]

# Changing anomaly value
Ecommerce['Region'].replace(['4orth','So3th'],['North','South'],inplace=True)

# Cross Checking its unique value
RG_count = Ecommerce['Region'].value_counts().reset_index()
RG_count.columns = ['Region','Total Count']
RG_count.sort_values(by='Total Count',ascending=False)

,Region,Total Count
0,Central,11117
1,South,6645
2,EMEA,5029
3,North,4785
4,Africa,4587
5,Oceania,3487
6,West,3203
7,Southeast Asia,3129
8,East,2848
9,North Asia,2338


### Quantity  Anomaly Feature Handling

There are values with both string type (disguised as numeric) and integer type. Have to replace the string to its appropriate type, integer.

In [25]:
# Replacing string to integer value
Ecommerce['Quantity'].replace(['1','2','3','4','5'],[1,2,3,4,5],inplace=True)

# Changing anomaly value
Ecommerce['Quantity'].replace(['abc'],[1],inplace=True)

# Cross Checking its unique value
QT_count = Ecommerce['Quantity'].value_counts().reset_index()
QT_count.columns = ['Quantity','Total Count']
QT_count.sort_values(by='Total Count',ascending=False)

,Quantity,Total Count
0,3.0,10307
1,1.0,10299
2,4.0,10278
3,2.0,10209
4,5.0,10196


In [26]:
# Locating Missing Value of Quantity
filt4 = Ecommerce['Quantity'].isnull()
Ecommerce.loc[filt4]

# Replacing Nan to 1
Ecommerce['Quantity'] = Ecommerce['Quantity'].fillna(value='1')

# Double Checking Missing and Blank values
QT_fill = Ecommerce['Quantity'].isnull().sum()
QT_blnk = Ecommerce['Quantity'].apply(lambda x : x == ' ').sum()

print('Quantity missing value:', QT_fill , 'and blank value :', QT_blnk)

Quantity missing value: 0 and blank value : 0


### Order Date MIssing Value Handling

Replacing missing value with the average day difference between ship date and order date. The average is 3 days. Just subtract the ship date in which the missing values located. We have the order day 20 april 2015

In [27]:
# Dropping Missing Value
Ecommerce['Order Date'] = Ecommerce['Order Date'].fillna(value='4/20/15')

# Double Checking Missing and Blank values
OT_fill = Ecommerce['Order Date'].isnull().sum()
OT_blnk = Ecommerce['Order Date'].apply(lambda x : x == ' ').sum()

print('Order Date missing value:', OT_fill , 'and blank value :', OT_blnk)

Order Date missing value: 0 and blank value : 0


### Customer ID Missing Value Handling

Simply drop this missing valueas it very unique to that corresponding customer.

In [28]:
# Dropping Missing Value
Ecommerce['Customer ID'] = Ecommerce['Customer ID'].dropna

# Double Checking Missing and Blank values
ID_fill = Ecommerce['Customer ID'].isnull().sum()
ID_blnk = Ecommerce['Customer ID'].apply(lambda x : x == ' ').sum()

print('Customer ID missing value:', ID_fill , 'and blank value :', ID_blnk)

Customer ID missing value: 0 and blank value : 0


### Discount Feature Handling

Fill the missing value with the most frequent value.

In [29]:
# Replacing string to integer value
Ecommerce['Discount'].replace(['0.01','0.02','0.03','0.04','0.05'],[0.01,0.02,0.03,0.04,0.05],inplace=True)

# Changing anomaly value
Ecommerce['Discount'].replace(['xxx'],[0.03],inplace=True)

# Cross Checking its unique value
DS_count = Ecommerce['Discount'].value_counts().reset_index()
DS_count.columns = ['Discount','Total Count']
DS_count.sort_values(by='Total Count',ascending=False)

,Discount,Total Count
0,0.03,10360
1,0.02,10262
2,0.04,10254
3,0.01,10242
4,0.05,10172


### Shipping Cost Feature Handling

Another odd / anomaly values from the feature, for this i simply drop the value.

In [30]:
# Specifying anomaly value
dp1 = Ecommerce['Shipping Cost'] == 'test'

# Dropping the specified row
Ecommerce.drop([535], inplace = True)

# Cross Checking its unique value
SP_count = Ecommerce['Shipping Cost'].value_counts().reset_index()
SP_count.columns = ['Shipping Cost','Total Count']
SP_count.sort_values(by='Total Count',ascending=False)

,Shipping Cost,Total Count
0,$2.10,1225
1,$1.80,1085
2,$1.60,1061
3,$1.70,1052
4,$11.20,930
...,...,...
148,$4.40,17
149,$5.10,17
150,$6.40,12
151,$6.80,7


### Sales Feature Handling

The same with the above feature, simply drop the value.

In [31]:
# Specifying anomaly value
dp2 = Ecommerce['Sales'] == '0.xf'

# Dropping the specified row
Ecommerce.drop([793], inplace = True)

# Cross Checking its unique value
SL_count = Ecommerce['Sales'].value_counts().reset_index()
SL_count.columns = ['Sales','Total Count']
SL_count.sort_values(by='Total Count',ascending=False).tail(5)

,Sales,Total Count
34,$33.00,221
35,$111.00,221
36,$222.00,221
37,$104.00,221
38,$199.00,221


### Cross Checking for Missing and Blank Value

In [ ]:
missing_data2 = Ecommerce.isnull().sum().reset_index()
missing_data2.columns = ['Features','Missing Values']
blank_data2 = Ecommerce.apply(lambda x : x == ' ').sum().reset_index()
blank_data2.columns = ['Features','Blank Values']
merged2 = pd.merge(missing_data2,blank_data2, on=['Features'])
merged2['Filling Factor'] = (Ecommerce.shape[0]-missing_data2['Missing Values'] + Ecommerce.shape[0]-blank_data2['Blank Values'])/Ecommerce.shape[0]*50
merged2.sort_values(by='Filling Factor',ascending=True)

### Select Features Transformation

Several features, which are in object type, should be transformed into numeric type, integer and float. Quantity and DIscount are the example.

### Quantity and Discount Transformation Type

In [32]:
# Change feature to integer type
Ecommerce['Quantity'] = Ecommerce['Quantity'].astype('int64')
Ecommerce['Discount'] = Ecommerce['Discount'].astype('float64')

# Feature Dtypes
twotype = Ecommerce[['Aging','Quantity','Discount']].dtypes.reset_index()
twotype.columns = ['Features','Type']
twotype

,Features,Type
0,Aging,float64
1,Quantity,int64
2,Discount,float64


### Feature Engineering

Making a new feature which is able to extract number fomr string. Sales, Profit and Shipping Cost are all string typed feature, but in the form US Dollar Nominal. With this new feature, the nominal can be extracted. Also adjust the type corrrespond this new feature, which is in numeric type.

### Sales USD, Profit USD and Shipping Cost USD  New Feature

In [33]:
# Creating New Sales USD, Profit USD and Shipping Cost USD Feature
Ecommerce['Sales USD'] = Ecommerce['Sales'].str.extract('([0-9][,.]*[0-9]*)')
Ecommerce['Profit USD'] = Ecommerce['Profit'].str.extract('([0-9][,.]*[0-9]*)')
Ecommerce['Shipping Cost USD'] = Ecommerce['Shipping Cost'].str.extract('([0-9][,.]*[0-9]*)')

# Changing each of those Feature to integer type
Ecommerce['Sales USD'] = Ecommerce['Sales USD'].astype('float64')
Ecommerce['Profit USD'] = Ecommerce['Profit USD'].astype('float64')
Ecommerce['Shipping Cost USD'] = Ecommerce['Shipping Cost USD'].astype('float64')

Ecommerce[['Sales','Sales USD','Profit','Profit USD','Shipping Cost','Shipping Cost USD']].head(5)

,Sales,Sales USD,Profit,Profit USD,Shipping Cost,Shipping Cost USD
0,$140.00,140.0,$46.00,46.0,$4.60,4.6
1,$211.00,211.0,$112.00,112.0,$11.20,11.0
2,$117.00,117.0,$31.20,31.0,$3.10,3.1
3,$118.00,118.0,$26.20,26.0,$2.60,2.6
4,$250.00,250.0,$160.00,160.0,$16.00,16.0


In [34]:
newtype = Ecommerce[['Sales USD','Profit USD','Shipping Cost USD']].dtypes.reset_index()
newtype.columns = ['Features','Type']
newtype

,Features,Type
0,Sales USD,float64
1,Profit USD,float64
2,Shipping Cost USD,float64


### Selecting and Dropping Features

Dropping very specific and unique value of Customer ID and Order ID, the same apllies to the previous 3 USD nominal value, which we no longer need because of the nwely created feature above already represents the value.

In [35]:
# Dropping ID features
Ecommerce.drop(['Order ID','Customer ID','Sales','Profit','Shipping Cost'],axis=1,inplace=True)

### Date Feature Engineering

Order Date Feature Extraction. We want to extract what day, month, weektype and Quarter year type of this date feature.

In [36]:
# Change to date type feature
Ecommerce['Order Date'] = pd.to_datetime(Ecommerce['Order Date'], format = '%m/%d/%y')
Ecommerce['Ship Date'] = pd.to_datetime(Ecommerce['Ship Date'], format = '%m/%d/%y')

# Date feature Engineering
Ecommerce['Order Days of Week'] = Ecommerce['Order Date'].dt.day_name()
Ecommerce['Order Months'] = Ecommerce['Order Date'].dt.month_name()

# Weekday/Weekend Feature Engineering
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Monday', 'Order Weektype'] = 'Weekday'  
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Tuesday', 'Order Weektype'] = 'Weekday'  
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Wednesday', 'Order Weektype'] = 'Weekday'  
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Thursday', 'Order Weektype'] = 'Weekday' 
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Friday', 'Order Weektype'] = 'Weekend'  
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Saturday', 'Order Weektype'] = 'Weekend' 
Ecommerce.loc[Ecommerce['Order Days of Week'] == 'Sunday', 'Order Weektype'] = 'Weekend'  

# Year Quarter Feature Engineering
Ecommerce.loc[Ecommerce['Order Months'] == 'January', 'Order Year Quarter'] = '1st Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'February', 'Order Year Quarter'] = '1st Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'March', 'Order Year Quarter'] = '1st Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'April', 'Order Year Quarter'] = '2nd Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'May', 'Order Year Quarter'] = '2nd Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'June', 'Order Year Quarter'] = '2nd Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'July', 'Order Year Quarter'] = '3rd Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'August', 'Order Year Quarter'] = '3rd Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'September', 'Order Year Quarter'] = '3rd Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'October', 'Order Year Quarter'] = '4th Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'November', 'Order Year Quarter'] = '4th Quarter'  
Ecommerce.loc[Ecommerce['Order Months'] == 'December', 'Order Year Quarter'] = '4th Quarter'  


# Dropping Prior Month Feature
Ecommerce.drop(['Months'],axis=1,inplace=True)

In [37]:
Ecommerce

,Order Date,Ship Date,Aging,Ship Mode,Product Category,Product,Quantity,Discount,Order Priority,Customer Name,Segment,City,State,Country,Region,Sales USD,Profit USD,Shipping Cost USD,Order Days of Week,Order Months,Order Weektype,Order Year Quarter
0,2015-11-09,2015-11-17,8.0,First Class,Auto & Accessories,Car Media Players,2,0.05,Medium,Lane Daniels,Consumer,Brisbane,Queensland,Australia,Oceania,140.0,46.0,4.6,Monday,November,Weekday,4th Quarter
1,2015-06-30,2015-07-02,2.0,First Class,Auto & Accessories,Car Speakers,3,0.03,Medium,Alvarado Kriz,Home Office,Berlin,Berlin,Germany,Central,211.0,112.0,11.0,Tuesday,June,Weekday,2nd Quarter
2,2015-12-05,2015-12-13,8.0,First Class,Auto & Accessories,Car Body Covers,5,0.01,Critical,Moon Weien,Consumer,Porirua,Wellington,New Zealand,Oceania,117.0,31.0,3.1,Saturday,December,Weekend,4th Quarter
3,2015-05-09,2015-05-16,7.0,First Class,Auto & Accessories,Car & Bike Care,2,0.05,High,Sanchez Bergman,Corporate,Kabul,Kabul,Afghanistan,Central Asia,118.0,26.0,2.6,Saturday,May,Weekend,2nd Quarter
4,2015-07-09,2015-07-18,9.0,First Class,Auto & Accessories,Tyre,1,0.04,Critical,Rowe Jackson,Corporate,Townsville,Queensland,Australia,Oceania,250.0,160.0,16.0,Thursday,July,Weekday,3rd Quarter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,2015-01-21,2015-01-27,6.0,Standard Class,Fashion,Sports Wear,5,0.04,Medium,Welch Fein,Corporate,Pasadena,Texas,United States,Central,85.0,17.0,1.7,Wednesday,January,Weekday,1st Quarter
51286,2015-06-22,2015-06-24,2.0,Standard Class,Fashion,Sports Wear,1,0.03,Medium,Martinez Arnett,Corporate,Harare,Harare,Zimbabwe,Africa,85.0,2.5,0.2,Monday,June,Weekday,2nd Quarter
51287,2015-01-01,2015-01-07,6.0,Standard Class,Fashion,Sports Wear,1,0.05,Medium,Mccoy Duston,Home Office,Townsville,Queensland,Australia,Oceania,85.0,0.8,0.1,Thursday,January,Weekday,1st Quarter
51288,2015-12-07,2015-12-14,7.0,Standard Class,Fashion,Sports Wear,3,0.04,Medium,Bentley Zypern,Consumer,Houston,Texas,United States,Central,85.0,28.0,2.8,Monday,December,Weekday,4th Quarter


It is seen we have all the informations we need from the extraction of the Order Date Feature.

### Export Cleaned Dataframe to CSV

Export it to newly cleaned csv file.

In [ ]:
Ecommerce.to_csv('Ecommerce_Cleaned.csv',index=False)